In [1]:
### imports
import numpy as np
#import tensorflow.keras as keras
import keras
from matplotlib import pyplot as plt
import os, sys
import copy
import pickle
import sys
sys.path.append(os.path.abspath('../'))
from keras2c import keras2c_main
from keras import backend as K


Using TensorFlow backend.


layers:  bidirectional rnn
         merge with different sizes
         conv3d
         pool3d
         seperable conv
         conv transpose
         depthwise conv
         crop3d
         pad3d
         upsampling1d
         upsampling2d
         upsampling3d
         locally connected 1d
         locally connected 2d
         time distributed
         
         
 im2col for convolution?
 static or const?
 broadcasting sizes for merge layers
 find the common dimension and add them that way?
 
 
 
 common API for all function calls
 figure out max array sizes?
 figure out keras h5 file format
 


In [2]:
a = keras.layers.Input((4,5))
b = keras.layers.TimeDistributed(keras.layers.Dense(8))(a)
model = keras.models.Model(a,b)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
weights = model.layers[1].get_weights()
weights[0].shape

(5, 8)

In [5]:
layer = model.layers[1]
layer.layer.__call__(K.constant(np.ones((layer.input_shape[2:]))[np.newaxis,:]))

<tf.Tensor 'dense_1/BiasAdd:0' shape=(1, 8) dtype=float32>

In [19]:
layer.get_input_at(0).name.split(':')[0]

'input_1'

In [21]:
layer.__class__.__name__

'TimeDistributed'

In [7]:
from keras2c.io_parsing import *

In [24]:
model.inputs[0].name

'input_1:0'

In [9]:
get_layer_io_names(layer)

(['input_1'], ['time_distributed_1'])

In [25]:
        inshp = (9, 7, 6, 3)
        alpha = 0.5
        a = keras.layers.Input(inshp)
        b = keras.layers.LeakyReLU(alpha=alpha)(a)
        model = keras.models.Model(inputs=a, outputs=b)


In [31]:
model.layers[1].output.name

'leaky_re_lu_1/LeakyRelu:0'

In [712]:
outputs.shape

(1, 5, 4)

In [716]:
def k2c_sub2idx(sub, shape, ndim):
  #/* converts from subscript to linear indices in row major order */
    idx = 0
    temp = 0;
    for i in range(ndim):
        temp = sub[i];
        for j in range(ndim-1,i,-1):
            temp *= shape[j]
        idx += temp;
    return int(idx)

def k2c_idx2sub(idx,shape,ndim):
    sub = np.zeros(ndim)
    for j in range(ndim-1,-1,-1):
        sub[j] = idx % shape[j]
        idx = idx // shape[j]
    return tuple(sub.astype(int))

def flip(a, axis):
    ndim = a.ndim
    shp = a.shape
    a = a.flatten()

    step = 1
    reduced_size = 1
    for i in shp[axis:]:
        reduced_size *= i
    threshold = int(reduced_size/2)
    jump = int(reduced_size)

    k=0
    while k<a.size:
        sub = list(k2c_idx2sub(k,shp,ndim))
        sub[axis] = shp[axis]-sub[axis]-1
        idx = k2c_sub2idx(sub,shp,ndim)
        temp = a[k]
        a[k] = a[idx]
        a[idx] = temp
        if (k+1) % jump >= threshold:
            k = (k + 1 -threshold + jump)
        else:
            k += step
    return a.reshape(shp)

In [593]:
np.random.randint(1,5)

3

In [717]:
for i in range(10):
    ndim = 5 #np.random.randint(1,6)
    ax = 0 #np.random.randint(ndim)
    shp = (5,10,1,1,1) #tuple(np.random.randint(1,25,ndim))
    print(ndim)
    print(shp)
    a = np.random.random(shp)
    b1 = np.flip(a,ax)
    b2 = flip(a,ax)
    print(np.max(np.abs(b1-b2)))

5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0
5
(5, 10, 1, 1, 1)
0.0


In [546]:
a

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14]],

       [[15, 16, 17],
        [18, 19, 20],
        [21, 22, 23],
        [24, 25, 26],
        [27, 28, 29]]])

In [547]:
b1

array([[[ 2,  1,  0],
        [ 5,  4,  3],
        [ 8,  7,  6],
        [11, 10,  9],
        [14, 13, 12]],

       [[17, 16, 15],
        [20, 19, 18],
        [23, 22, 21],
        [26, 25, 24],
        [29, 28, 27]]])

In [548]:
b2

array([[[ 2,  1,  0],
        [ 5,  4,  3],
        [ 8,  7,  6],
        [11, 10,  9],
        [14, 13, 12]],

       [[17, 16, 15],
        [20, 19, 18],
        [23, 22, 21],
        [26, 25, 24],
        [29, 28, 27]]])

In [549]:
np.max(np.abs(b1-b2))

0

# # model testing

In [ ]:
shape1 = (None,5,4,2)
shape2 = (None,5,4,1)
output_shape = list(shape1[:-len(shape2)])
for i, j in zip(shape1[-len(shape2):], shape2):
    if i is None or j is None:
        output_shape.append(None)
    elif i == 1:
        output_shape.append(j)
    elif j == 1:
        output_shape.append(i)
    else:
        if i != j:
            raise ValueError('Operands could not be broadcast '
                             'together with shapes ' +
                             str(shape1) + ' ' + str(shape2))
        output_shape.append(i)
set([shape1,shape2]) # check if all inputs are the same shape

In [ ]:
convert_sequential_to_model(model).layers

In [ ]:
model.layers

In [ ]:
def convert_sequential_to_model(model):
    """Convert a sequential model to the underlying functional format"""
    if type(model).__name__ == 'Sequential':
        if hasattr(model, '_inbound_nodes'):
            inbound_nodes = model._inbound_nodes
        elif hasattr(model, 'inbound_nodes'):
            inbound_nodes = model.inbound_nodes
        else:
            raise ValueError('can not get (_)inbound_nodes from model')
        # Since Keras 2.2.0
        if model.model == model:
            input_layer = keras.layers.Input(batch_shape=model.layers[0].input_shape)
            prev_layer = input_layer
            for layer in model.layers:
                prev_layer = layer(prev_layer)
            funcmodel = keras.models.Model([input_layer], [prev_layer])
            model = funcmodel
        else:
            model = model.model
        if hasattr(model, '_inbound_nodes'):
            model._inbound_nodes = inbound_nodes
        elif hasattr(model, 'inbound_nodes'):
            model.inbound_nodes = inbound_nodes
    assert model.layers
    for i in range(len(model.layers)):
        if type(model.layers[i]).__name__ in ['Model', 'Sequential']:
            model.layers[i] = convert_sequential_to_model(model.layers[i])
    return model
